In [80]:
import numpy as np
import pandas as pd
import os, re
import scipy
import csv
from collections import Counter
import pickle
from sklearn.metrics.cluster import *
from sklearn.metrics import hamming_loss
import xgboost as xgb
from xgboost import plot_importance

In [ ]:
import features_evolu
import features_public
import features_regu
import features_stru

In [12]:
# %load_ext jupyternotify
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### Data input

In [214]:
df=pd.read_csv('Example.Input.txt',sep='\t')

In [215]:
df['AAb']=df['AAchange'].apply(lambda x:x[0])
df['AAa']=df['AAchange'].apply(lambda x:x[-1])

### Import cache data

In [95]:
for pkl in [x for x in os.listdir('cachedata/') if x.split('.')[-1]=='pkl']:
    dictN=pkl.split('.')[0]
    with open('cachedata/%s'%pkl,'rb') as tf:
        globals()[dictN]=pickle.load(tf)

### Obtain functional score of nsSNVs based on dbNSFP

In [153]:
with open('cachedata/dfIn/example.vepIn','w') as w:
    for i in df[['chr','chrl','NNchange','AAchange']].drop_duplicates().index:
        line=df[['chr','chrl','NNchange','AAchange']].loc[i]
        w.write(' '.join(line.astype(str)).replace('/',' ')+'\n')

* Download dbNSFP v4.3 from http://database.liulab.science/dbNSFP
    * CITATION: Liu X, Li C, Mou C, Dong Y, and Tu Y. 2020. dbNSFP v4: a comprehensive database of transcript-specific functional predictions and annotations for human nonsynonymous and splice-site SNVs. Genome Medicine. 12:103.
    
* Run dbNSFP v4.3
    * java -Xmx4g search_dbNSFP43a -i example.vepIn -o example.vepOut -w 1-9,12-18,38-43,100-101,141-142,155-157,162-163
    * To speed up dbNSFP v4.3, you can use the -c parameter to limit the result to a specific chromosome, for example, you can use -c 13 for chromosome 13
* Place the output result 'example.vepOut' into the path 'cachedata/dfIn/'.

In [147]:
def get_vep(df, vepPath):
    df['dnnsfp'] = df.chr.astype(str) + '_' + df.chrl.astype(
        str
    ) + '_' + df.NNchange + '_' + df.AAchange  #+ '_' + df.snvLoc.astype(str)

    dnnsfp = pd.read_csv(vepPath, sep='\t')
    dnnsfp['dnnsfp'] = dnnsfp['#chr'].astype(str) + '_' + dnnsfp[
        'pos(1-based)'].astype(str) + '_' + dnnsfp.ref.astype(
            str) + '/' + dnnsfp.alt.astype(str) + '_' + dnnsfp.aaref.astype(
                str) + '/' + dnnsfp.aaalt.astype(
                    str)  # + '_' + dnnsfp.aapos.astype(str)

    # dnnsfp['Uniprot'] = dnnsfp.Uniprot_acc.str.split('-', expand=True)[0]
    
    dnnsfp = dnnsfp[[
        'phyloP17way_primate', 'phyloP17way_primate_rankscore',
        'BayesDel_addAF_score', 'BayesDel_addAF_rankscore',
        'integrated_fitCons_score', 'integrated_fitCons_rankscore',
        'GERP++_RS', 'GERP++_RS_rankscore', 'dnnsfp'
    ]].drop_duplicates().replace('.',np.nan)
    
    df = pd.merge(df,
                  dnnsfp,
                  left_on=['dnnsfp'],
                  right_on=['dnnsfp'],
                  how='left')
    return df

In [217]:
## 'GERP_RS_rankscore', 'phyloP17way_primate_rankscore', 'Integrated_fitCons_rankscore', 'BayesDel_addAF_rankscore',
df=get_vep(df, 'cachedata/dfIn/example.vepOut')

### Calculate VIP (variant impact on phosphorylation) features

In [99]:
def get_Feature(df):
    
    ### 'feature_PWM_refpep', 'feature_PWM_varpep', 'feature_PWM_maxdis',
    df=features_regu.get_PWM(df)
    
    ### 'feature_paxdb_log10'
    df=features_public.getPaxdb(df)

    # ### 'feature_distance_abs', 'feature_length',
    df=features_stru.getF_1d(df)

    ## 'PSIPRED_ss3H', 'PSIPRED_ss3E', 'PSIPRED_ss3C', 'ASAquick_asa', 'feature_psitesnv_LCD', 'feature_psitesnv_IDR',
    df=features_stru.getF_2d(df)

    ### 'feature_dist_3d',
    df=features_stru.getF_3d(df)

    ### 'feature_psite_ptmage0', 'feature_psite_ptmage3', 'feature_pubmed_n',
    df=features_evolu.get_ptmage_pubmed(df)

    ### 'feature_netpho_max_all', 'feature_netpho_max_kin', 'feature_netpho_max_nokin',
    df=features_regu.get_netpho(df)

    ### 'feature_psitesnv_ptm21', 'feature_psitesnv_nbt21', 'feature_psitesnv_nbt_score',
    df=features_regu.get_ptm21_nbt(df)
    df=features_regu.get_Ubi21(df)

    ### aaindex_SNVbox
    df=features_stru.get_AAindex(df)
    
    ### 'site_coevolve', 
    df=features_evolu.get_coevolve(df)

    ### 'feature_sift_psite', 'feature_provean_psite',
    df=features_evolu.get_sift(df)

    ###  Determine whether the phosphorylation site is a known phosphorylation site in PhosphositePLUS or as reported by Ochoa et al.
    df=features_public.get_psite_existed(df)
    
    return df

In [218]:
df=get_Feature(df)

### Predict VIP events based on VIPpred

In [122]:
featureList = [
    'feature_paxdb_log10',
    'feature_length',
    'feature_distance_abs',
    'ASAquick_asa',
    'PSIPRED_ss3H', 'PSIPRED_ss3E', 'PSIPRED_ss3C',
    'feature_LCD', 'feature_IDR',
    'dist_3d',
    'feature_netpho_max_all', 'feature_netpho_max_kin',
    'feature_psitesnv_nbt21', 
    'feature_psitesnv_ptm21',
    'AAPolarity', 'AAVolume', 'AAHydrophobicity', 'AAGrantham',
    'site_coevolve',
    'feature_PWM_varpep', 'feature_PWM_refpep', 
    'phyloP17way_primate',
    'BayesDel_addAF_score',
    'integrated_fitCons_score',
    'GERP++_RS',
    'feature_sift_psite',
    'feature_pubmed_MS_LIT', 'feature_pubmed_LT_LIT',
    'feature_psite_ptmage0', 'feature_psite_ptmage3',
    'feature_psite_existed','feature_psite_ubi21'
]

In [125]:
def get_predLabel(df,model,col1,col2,features=featureList):
    
    ## gain loss都预测一遍
    cachedf=df.fillna({
        'feature_pubmed_MS_LIT': 0,
        'feature_pubmed_LT_LIT': 0,
        'feature_psite_ptmage0': 0,
        'feature_psite_ptmage3': 0,
        'feature_psitesnv_compbias': 0,
        'feature_LCD': 0,
        'feature_IDR': 0,
        'feature_psitesnv_nbt21': 0,
        'feature_psitesnv_ptm21': 0,
    })
    
    Xtest = cachedf[features].values
    probas = model.predict_proba(Xtest)
    print('%s get'%col1)
    label = model.predict(Xtest)
    print('%s get'%col2)
    cachedf[col1]=probas[:, 1]
    cachedf[col2]=label
    
    return cachedf

In [126]:
def getLabel(row,col1,col2,col1s,col2s):
    if row[col1]==0 and row[col2]==0:
        return 'pairNoimpact'
    elif row[col1]==0 and row[col2]==1:
        return 'pairLoss'
    elif row[col1]==1 and row[col2]==0:
        return 'pairGain'
    elif row[col1s]>row[col2s]:
        return 'pairGain'
    elif row[col1s]<row[col2s]:
        return 'pairLoss'
    else:
        print(row.index)

In [123]:
## Load VIPpred model
modelG=pickle.load(open('modelGain.pickle.dat','rb'))
modelL=pickle.load(open('modelLoss.pickle.dat','rb'))

In [219]:
df=get_predLabel(df,modelG,'gainScore','gainLabel')
df=get_predLabel(df,modelL,'lossScore','lossLabel')
df['VIP_Label']=df.apply(lambda row:getLabel(row,'gainLabel','lossLabel','gainScore','lossScore'),axis=1)

gainScore get
gainLabel get
lossScore get
lossLabel get


In [220]:
## Output
df.to_csv('Example.Output.txt',sep='\t')